In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from random import shuffle

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
images_path = os.listdir('../Cat vs Dog (Tensorflow)/train')

In [3]:
def generate_train_data():
    train_X = []
    label_X = []
    for file in images_path:
        img = cv2.imread(os.path.join('../Cat vs Dog (Tensorflow)/train',file),0)
        img = cv2.resize(img,(56,56))
        if(file[0:3]=='cat'):
            train_X.append([img,'0'])
        else:
            train_X.append([img,'1'])
    shuffle(train_X)    
    return train_X    

In [4]:
data = generate_train_data()

In [5]:
train_X = [data[i][0] for i in range(len(data))]

In [6]:
train_X = np.asarray(train_X)

In [7]:
train_y = [data[i][1] for i in range(len(data))]

In [9]:
train_y = np.asarray(train_y)

In [10]:
train_X.shape, train_y.shape

((25000, 56, 56), (25000,))

In [11]:
train_X = train_X.reshape(-1,56,56,1)

In [13]:
train_X.shape, train_y.shape

((25000, 56, 56, 1), (25000,))

In [14]:
train_y = pd.get_dummies(train_y)

In [15]:
train_y.shape

(25000, 2)

In [16]:
epochs = 20
lr = 0.01
batch_size = 50

In [17]:
n_classes = 2

In [18]:
x = tf.placeholder(tf.float32,[None,56,56,1])
y = tf.placeholder(tf.float32,[None,2])

In [19]:
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)
def max_pool(x,pool=2):
    return tf.nn.max_pool(x,ksize=[1,pool,pool,1],strides=[1,pool,pool,1],padding='SAME')

In [20]:
weight={
    'wc1':tf.get_variable('w0',shape=(3,3,1,32),initializer=tf.contrib.layers.xavier_initializer()),
    'wc2':tf.get_variable('w1',shape=(3,3,32,64),initializer=tf.contrib.layers.xavier_initializer()),
    'wc3':tf.get_variable('w2',shape=(3,3,64,64),initializer=tf.contrib.layers.xavier_initializer()),
    'wc4':tf.get_variable('w3',shape=(3,3,64,128),initializer=tf.contrib.layers.xavier_initializer()),
    'wd0':tf.get_variable('wde0',shape=(4*4*128,128),initializer=tf.contrib.layers.xavier_initializer()),
    'wd1':tf.get_variable('wde1',shape=(128,2),initializer=tf.contrib.layers.xavier_initializer())
}

biases={
    'bc1':tf.get_variable('b0',shape=(32),initializer=tf.contrib.layers.xavier_initializer()),
    'bc2':tf.get_variable('b1',shape=(64),initializer=tf.contrib.layers.xavier_initializer()),
    'bc3':tf.get_variable('b2',shape=(64),initializer=tf.contrib.layers.xavier_initializer()),
    'bc4':tf.get_variable('b3',shape=(128),initializer=tf.contrib.layers.xavier_initializer()),
    'bd0':tf.get_variable('bde0',shape=(128),initializer=tf.contrib.layers.xavier_initializer()),
    'bd1':tf.get_variable('bde1',shape=(2),initializer=tf.contrib.layers.xavier_initializer())
}

In [21]:
def conv_net(x,weight,biases):
    conv1 = conv2d(x,weight['wc1'],biases['bc1'])
    conv1 = max_pool(conv1,2)
    
    conv2 = conv2d(conv1,weight['wc2'],biases['bc2'])
    conv2 = max_pool(conv2,2)
    
    conv3 = conv2d(conv2,weight['wc3'],biases['bc3'])
    conv3 = max_pool(conv3,2)
    
    conv4 = conv2d(conv3,weight['wc4'],biases['bc4'])
    conv4 = max_pool(conv4,2)
    
    fc1 = tf.reshape(conv4, [-1, weight['wd0'].get_shape().as_list()[0]])
    
    fc1 = tf.add(tf.matmul(fc1, weight['wd0']), biases['bd0'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weight['wd1']), biases['bd1'])
    
    return out
    

In [22]:
pred = conv_net(x,weight,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(lr).minimize(cost)

In [23]:
correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [24]:
init = tf.global_variables_initializer()

In [25]:
with tf.Session() as sess:
    sess.run(init)
    
    training_loss = []
    test_loss = []
    training_accuracy = []
    test_accuracy = []
    
    for _ in range(epochs):
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]
            
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
            loss,acc = sess.run([cost,accuracy],feed_dict={x:batch_x,y:batch_y})
            
            print("Loss= " + \
                          "{:.6f}".format(loss) + ", Training Accuracy= " + \
                          "{:.5f}".format(acc))
        
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : y_test})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))

Loss= 1866.195679, Training Accuracy= 0.50000
Loss= 19.524324, Training Accuracy= 0.58000
Loss= 9.295268, Training Accuracy= 0.56000
Loss= 13.576677, Training Accuracy= 0.54000
Loss= 8.592461, Training Accuracy= 0.50000
Loss= 1.155270, Training Accuracy= 0.48000
Loss= 1.265951, Training Accuracy= 0.42000
Loss= 0.854215, Training Accuracy= 0.56000
Loss= 0.799149, Training Accuracy= 0.44000
Loss= 0.698455, Training Accuracy= 0.54000
Loss= 0.723204, Training Accuracy= 0.46000
Loss= 0.670634, Training Accuracy= 0.58000
Loss= 0.629426, Training Accuracy= 0.62000
Loss= 0.652277, Training Accuracy= 0.60000
Loss= 0.647255, Training Accuracy= 0.58000
Loss= 0.660915, Training Accuracy= 0.62000
Loss= 0.703197, Training Accuracy= 0.56000
Loss= 0.696395, Training Accuracy= 0.52000
Loss= 0.686181, Training Accuracy= 0.54000
Loss= 0.669103, Training Accuracy= 0.54000
Loss= 0.681022, Training Accuracy= 0.56000
Loss= 0.684732, Training Accuracy= 0.54000
Loss= 0.680675, Training Accuracy= 0.54000
Loss= 

KeyboardInterrupt: 